<a href="https://colab.research.google.com/github/Guythetwo/test/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# โหลดข้อมูล
data = pd.read_csv('/content/drive/MyDrive/ML/dataset.csv')

# ตรวจสอบข้อมูลที่หายไป
#print(data.isnull().sum())

# แปลงตัวแปรเพศ (Sex) ให้เป็นตัวเลข
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])

# เลือกคุณสมบัติ (Features) และเป้าหมาย (Target)
X = data.drop(['Sex', 'age level'], axis=1)
y = data['Sex']

# แบ่งข้อมูลเป็นชุดฝึกอบรมและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# กำหนดพารามิเตอร์สำหรับการค้นหา
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# สร้างโมเดล Random Forest และทำการค้นหาพารามิเตอร์ที่ดีที่สุด
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# ใช้โมเดลที่ดีที่สุดในการทำนายผลบนชุดทดสอบ
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# ประเมินประสิทธิภาพของโมเดล
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Accuracy: 0.5411061285500748
Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.36      0.42       225
           1       0.64      0.80      0.71       200
           2       0.47      0.50      0.48       244

    accuracy                           0.54       669
   macro avg       0.54      0.55      0.54       669
weighted avg       0.53      0.54      0.53       669



In [8]:
import joblib

joblib.dump(best_model, 'best_model.joblib')

['best_model.joblib']

In [22]:
import joblib
import numpy as np
# โหลดโมเดลจากไฟล์
loaded_model = joblib.load('best_model.joblib')
label = np.array(['female','I', 'male'])
# สร้างข้อมูลสำหรับทำนาย
X_new = np.array([
    [0.55,0.445,0.125,0.672,0.288,0.1365,0.21,11],
    [0.305,0.225,0.07,0.1485,0.0585,0.0335,0.045,7]
    ])
# ใช้โมเดลที่โหลดมาทำนายผล
y_pred = loaded_model.predict(X_new)
#X_new
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array(['Adult', 'young'], dtype=object)

In [27]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# โหลดข้อมูลที่ไม่เคยเห็น
unseen_data = pd.read_csv('/content/drive/MyDrive/ML/unseen.csv')

# ตรวจสอบข้อมูลที่หายไปและจัดการกับมัน (เช่นเดียวกับข้อมูลฝึกอบรม)
for column in unseen_data.select_dtypes(include=['float64', 'int64']).columns:
    unseen_data[column].fillna(unseen_data[column].mean(), inplace=True)

for column in unseen_data.select_dtypes(include=['object']).columns:
    unseen_data[column].fillna(unseen_data[column].mode()[0], inplace=True)

# แปลงตัวแปรเพศ (Sex) ให้เป็นตัวเลข
le = LabelEncoder()
unseen_data['Sex'] = le.fit_transform(unseen_data['Sex'])

# เตรียมข้อมูล (เช่นเดียวกับการเตรียมข้อมูลฝึกอบรม)
X_unseen = unseen_data.drop(['Sex', 'age level'], axis=1)
y_true = unseen_data['Sex']

# โหลดโมเดลที่ฝึกอบรมแล้ว
model = joblib.load('best_model.joblib')

# ทำนายผลด้วยข้อมูลที่ไม่เคยเห็น
y_pred = model.predict(X_unseen)

# ประเมินผลลัพธ์
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)

print(f'Accuracy on unseen data: {accuracy}')
print('Classification Report on unseen data:')
print(report)


Accuracy on unseen data: 1.0
Classification Report on unseen data:
              precision    recall  f1-score   support

       Adult       1.00      1.00      1.00       581
         old       1.00      1.00      1.00        75
       young       1.00      1.00      1.00       180

    accuracy                           1.00       836
   macro avg       1.00      1.00      1.00       836
weighted avg       1.00      1.00      1.00       836

